# Protein structure superposing



## 3) **Superposing with Python**

Gemmi is an efficient parser for the `mmCIF` file format. We will be using it here to parse in structures from the PDBe we have saved locally. Once loaded into the script, we will explore different protein structure superposition tools. 

In [1]:
import subprocess
import pathlib
import gemmi
import Bio

### 3.1) Loading and saving structure files

Structure files, typically in PDB or mmCIF formats, can be parsed into Python scripts using the built-in read-write functions, such as `open()` and `write()`. However, tools have been developed to make this process quick and easy. We will be focusing on the file parsing module Gemmi as it is relatively quick and has good support for the modern mmCIF file format. Another set of functions for parsing structure files into Python is included in the popular Biopython library, which tends to focus on files in the PDB format. 


#### **Gemmi**

[Gemmi](https://gemmi.readthedocs.io/en/latest/) is a modern Python and C++ library containing many tools for performing common tasks in structural biology. It is well suited to handling mmCIF (PDBx/mmCIF) file formats, which are organised into blocks and loops of data.

To load a mmCIF file:

In [ ]:
from gemmi import cif

# Location of saved structure file
path_mmcifs = "examples_mmcif/"
path_6mka = str( path_mmcifs + "6mka.cif" )

# Load file into program
mmcif_6mka = cif.read_file( path_6mka ).sole_block()

print(mmcif_6mka)

To extract information from the parsed mmCIF file, we can use the `find()` method, where we provide the blocks and loops of data we want to access. For a complete list of all data blocks and loops, refer to the [file format documentation](https://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Categories/index.html) provided by the [wwPDB](http://www.wwpdb.org/). 

As an introductory example, the code below will extract the atom type, atom label, atom coordinates, occupancy and chain ID (author stated).

In [ ]:
import numpy as np
import pandas as pd

# Extract data
mmcif_table =  mmcif_6mka.find(
    "_atom_site.",
    [
        "group_PDB",
        "label_atom_id",
        "Cartn_x",
        "Cartn_y",
        "Cartn_z",
        "occupancy",
        "auth_asym_id"
    ],
    )

# Convert to Numpy array
# ... not strictly necessary but improves runtime performance and data handling
mmcif_array = np.asarray(mmcif_table)

# Convert to Pandas DataFrame
# ... especially useful for data handling and selecting rows
mmcif_df = pd.DataFrame(
    mmcif_table,
    columns=[
        "group_PDB",
        "label_atom_id",
        "Cartn_x",
        "Cartn_y",
        "Cartn_z",
        "occupancy",
        "auth_asym_id"
        ]
    )

# Extract only the CA atoms
ca_atoms = mmcif_df.loc[mmcif_df["label_atom_id"] == "CA"]
print(ca_atoms)


Gemmi can also load structure files as coordinate objects, a feature especially useful for superpositioning. We will be using this method of parsing for superposing structures but the former approach can be used to extract any information stored in the mmCIF file. 

In [ ]:
# Load example structure into script
model_original = gemmi.read_structure('./examples_mmcif/1lzh_updated.cif') # PDB format accepted

print(model_original.ncs)


model = model_original[0]

# Assign static chain
static = model['A'].get_polymer()
# Assign mobile chains
mobile = model['B'].get_polymer()
ptype = static.check_polymer_type()     # Useful when not already known

superposed = gemmi.calculate_superposition(
    static,               # Structure 1
    mobile,               # Structure 2
    ptype,                  # The type of macromolecule
    gemmi.SupSelect.CaP     # Select only CAs
    )

print("RMSD = ", superposed.rmsd)

# Some useful transformations
superposed_matrix = superposed.transform.mat
superposed_vector = superposed.transform.vec

superposed_matrix = np.asarray(superposed_matrix)
superposed_vector = np.asarray(superposed_vector)

print(superposed_matrix)
print(superposed_vector)




### 3.1) Superposing in Python

#### **Biopython**
One approach to superposing structures programatically is use the popular Python package _Biopython_. The package contains three structural alignment tools, `SVDSuperimposer`, `Superimposer` and `CEAligner`. Each tool implements a different algorithm that attempts to minimise the root mean square deviation (RMSD) between all atoms for two given protein structures. A quick summary for each is detailed below:

 - `SVDSuperimposer`: 
 - `Superimposer`:
 - `CEAligner`:


 

In [ ]:
import Bio.PDB
import numpy

pdb_code = "1JOY"
pdb_filename = "%s.pdb" % pdb_code
pdb_out_filename = "%s_aligned.pdb" % pdb_code

seq_str = 'MAAGVKQLADDRTLLMAGVSHDLRTPLTRIRLATEMMSEQDGYLAESINKDIEECNAIIEQFIDYLR'
use_str = '-----------RTLLMAGVSHDLRTPLTRIRLATEMMSEQDGYLAESINKDI---------------'
use = [letter<>"-" for letter in use_str]
assert len(use) == len(seq_str)

In [ ]:
    ref_atoms = []
    alt_atoms = []
    for (ref_chain, alt_chain) in zip(ref_model, alt_model) :
        for ref_res, alt_res, amino, allow in zip(ref_chain, alt_chain, seq_str, use) :
            assert ref_res.resname == alt_res.resname
            assert ref_res.id      == alt_res.id
            assert amino == Bio.PDB.Polypeptide.three_to_one(ref_res.resname)
            if allow :
                #CA = alpha carbon
                ref_atoms.append(ref_res['CA'])    
                alt_atoms.append(alt_res['CA'])

In [ ]:
print("Loading PDB file %s" % pdb_filename)
structure = Bio.PDB.PDBParser().get_structure(pdb_code, pdb_filename)

print("Everything aligned to first model...")
ref_model = structure[0]
for alt_model in structure :
    #Build paired lists of c-alpha atoms, ref_atoms and alt_atoms
    #[code shown later]

    #Align these paired atom lists:
    super_imposer = Bio.PDB.Superimposer()
    super_imposer.set_atoms(ref_atoms, alt_atoms)

    if ref_model.id == alt_model.id :
        #Check for self/self get zero RMS, zero translation
        #and identity matrix for the rotation.
        assert numpy.abs(super_imposer.rms) < 0.0000001
        assert numpy.max(numpy.abs(super_imposer.rotran[1])) < 0.000001
        assert numpy.max(numpy.abs(super_imposer.rotran[0]) - numpy.identity(3)) < 0.000001
    else :
        #Update the structure by moving all the atoms in
        #this model (not just the ones used for the alignment)
        super_imposer.apply(alt_model.get_atoms())

    print("RMS(first model, model %i) = %0.2f" % (alt_model.id, super_imposer.rms) )

#### **Gemmi**



### 3.2) Command line tools via Python

Some tools are often only available as command line applications, meaning they lack a dedicated Python (or any other programming language) API. This prevents us from using the code for these tools directly in our code. However, we can still use these applications through the command line using the `subprocess` module. You will sometimes see the `os` module used for functionally similar purposes. Which you decide to use is often a matter of preference. We will be considering the `subprocess` module as it is rich in features relative to `os`. 

`subprocess` allows us to send commands to the terminal through Python. Take a look at the general example below:



In [2]:
import subprocess

# Define the command to run in the terminal
path = '.'
command = [f"ls -ltr {path}"]       # ls -ltr .

# Ensure results are formatted as strings
args = {
    "shell" : True, 
    "encoding" : "utf-8"
    }

# Send the command to the terminal for execution
results = subprocess.check_output(command, **args).splitlines()

# Display the results, line-by-line, from the command
for line in results:
    print(line)

total 17280
-rw-r--r--  1 jellaway  staff   405166 May  5 15:12 sup.out
-rw-r--r--  1 jellaway  staff   405166 May  5 15:19 superpose_example_output.pdb
-rw-r--r--  1 jellaway  staff   765940 May  5 16:01 gesamt_example_output.pdb
-rw-r--r--  1 jellaway  staff  3628476 May 14 14:55 1joy.pdb
drwxr-xr-x  8 jellaway  staff      256 May 14 14:55 figures
-rw-r--r--  1 jellaway  staff     5739 May 14 17:09 superposition_1_web.ipynb
-rw-r--r--  1 jellaway  staff  3603199 Jun  7 09:09 1joy_aligned.pdb
drwxr-xr-x  8 jellaway  staff      256 Jun  7 09:09 examples_mmcif
drwxr-xr-x  9 jellaway  staff      288 Jun  7 09:09 examples_pdb
-rw-r--r--  1 jellaway  staff    11616 Jun  7 09:55 superposition_2_local.ipynb
-rw-r--r--  1 jellaway  staff    14846 Jun  7 10:29 superposition_3_python.ipynb


Providing you are in the same directory in which this notebook is running, executing the command `ls -ltr .` will give you exactly the same result. 

To simply run a command in the terminal without saving the results, use:

In [8]:
import os

# Define the command to run in the terminal
path = '.'
command = f"ls -ltr {path}"       # ls -ltr .

# Execute the command in the terminal. Results not saved
results = os.system(command)

total 17296
-rw-r--r--  1 jellaway  staff   405166 May  5 15:12 sup.out
-rw-r--r--  1 jellaway  staff   405166 May  5 15:19 superpose_example_output.pdb
-rw-r--r--  1 jellaway  staff   765940 May  5 16:01 gesamt_example_output.pdb
-rw-r--r--  1 jellaway  staff  3628476 May 14 14:55 1joy.pdb
drwxr-xr-x  8 jellaway  staff      256 May 14 14:55 figures
-rw-r--r--  1 jellaway  staff     5739 May 14 17:09 superposition_1_web.ipynb
-rw-r--r--  1 jellaway  staff  3603199 Jun  7 09:09 1joy_aligned.pdb
drwxr-xr-x  8 jellaway  staff      256 Jun  7 09:09 examples_mmcif
drwxr-xr-x  9 jellaway  staff      288 Jun  7 09:09 examples_pdb
-rw-r--r--  1 jellaway  staff    11616 Jun  7 09:55 superposition_2_local.ipynb
-rw-r--r--  1 jellaway  staff    24476 Jun  7 10:04 superposition_3_python.ipynb


_NB_: `ls` lists all of the files and subdirectories in a specified folder, `-ltr` are flags that modify the command's behaviour, and `.` tells the command to work on _this_ directory. `-ltr` is equivalent to `-l -t -r`, which are flags to tell `ls` to provide extended file/directory information, order the results by date modified, and reverse this order to place most recent files at the bottom, respecitvely. 

We can, therefore, implement `subprocess` commands in Python scripts to run superposition applications. Of course, this package is not exclusive to superposition and can be used to run any command line application automatically.

The code below will serially superpose the structures in the `examples_mmcif` directory, using SSM and GESAMT.  

In [4]:
import subprocess

# Define the commands for running superposition applications
ssm = "superpose"
gesamt = "gesamt"

# Strings that will not change in script
path_structures = "./examples_mmcif/"
static_struct = "6mka.cif"
path_static = f"{path_structures}{static_struct}"

# Loop over the structures for superposition
for mobile_struct in ["6mkf.cif", "6mkg.cif", "6mkj.cif"]:

    # Path to mobile structure in superposition
    path_mobile = f"{path_structures}{mobile_struct}"

    # Out file names
    save_file_ssm = f"ssm_{mobile_struct[:4]}_to_{static_struct[:4]}.pdb"
    save_file_gesamt = f"gesamt_{mobile_struct[:4]}_to_{static_struct[:4]}.pdb"

    # SSM command ready
    command_ssm = [ssm, path_static, path_mobile, "-o", save_file_ssm]
    command_ssm = f"{ssm} {path_static} {path_mobile} -o {save_file_ssm}"
    # GESAMT command ready
    command_gesamt = f"{gesamt} {path_static} {path_mobile} -o {save_file_gesamt}"
    print(command_ssm)

    # Execute SSM command
    os.system(command_ssm)
    # Execute GESAMT command
    os.system(command_gesamt)

superpose ./examples_mmcif/6mka.cif ./examples_mmcif/6mkf.cif -o ssm_6mkf_to_6mka.pdb
superpose ./examples_mmcif/6mka.cif ./examples_mmcif/6mkg.cif -o ssm_6mkg_to_6mka.pdb
superpose ./examples_mmcif/6mka.cif ./examples_mmcif/6mkj.cif -o ssm_6mkj_to_6mka.pdb


sh: superpose: command not found
sh: superpose: command not found
sh: superpose: command not found


If you receive a `command not found` error message, instead run the `ssm_gesamt_via_python.py` script, which contains exactly the same code. 

### **Practice exercise 3:** Serial superposition

Using your results collected from the initial search of 5igh against one or of your chosen databases from exercise 1), write a Python script to superpose the models using either GESAMT or SSM. It might be useful to delegate some of the work across the group, allowing you to compare the SSM and GESAMT results to see if they agree. 

Once superposed, you can load your results into PyMol or Mol* and begin to identify common ligand-binding features between structures. 